HERE I AM GOING TO DO
- **STEP 1:** Parse the metadata
&
- **STEP 4:** Explore them (from the First notebook's list of goals)

In [113]:
import os
import pandas as pd
import re

# Specify the directory containing the files
input_folder = '/content/sample_data/THESIS/'

# Loop through each file in the directory
for filename in os.listdir(input_folder):
    if filename.endswith('.txt'): # because our files are in this format
        # Construct the full file path
        input_file = os.path.join(input_folder, filename)

        # Construct the output name by removing '.txt' and adding '_parsed_metadata.tsv'
        output_file = os.path.join(input_folder, f"{filename[:-4]}_parsed_metadata.tsv")

        # Initialize 2 lists: one to store keys and the second one values
        keys = []
        values = []

        # Open the series matrix file
        with open(input_file, 'r') as file:
            for line in file:
                # Check if the line starts with '!' to identify metadata
                if line.startswith('!'):
                    # Remove leading '!' and split by tab
                    parts = line[1:].strip().split('\t', 1)  # Split into key and value
                    if len(parts) == 2:
                        key, value = parts
                        keys.append(key)

                        # For other lines, split by tabs
                        value_parts = value.split('\t')
                        values.append(value_parts)

                    else:
                        # Handle any other lines that don't split as expected
                        keys.append(parts[0])
                        values.append([])  # Empty list for missing values

        # Create a DataFrame from keys and values
        metadata_df = pd.DataFrame({'Key': keys})

        # Expand the 'Value' list into separate columns
        value_columns = pd.DataFrame(values)

        # Concatenate key and value columns
        result_df = pd.concat([metadata_df, value_columns], axis=1)

        # Save the result as a TSV file
        result_df.to_csv(output_file, sep='\t', index=False)

        print(f"Metadata for {filename} has been saved to {output_file}")


Metadata for GSE85544_series_matrix.txt has been saved to /content/sample_data/THESIS/GSE85544_series_matrix_parsed_metadata.tsv
Metadata for GSE85543_series_matrix.txt has been saved to /content/sample_data/THESIS/GSE85543_series_matrix_parsed_metadata.tsv




---


So now it is time to start the metadata file processing, because I have noticed that certain things in both tsv files are not as expected

---



In [ ]:
# Loop through each file in the directory
for filename in os.listdir(input_folder):
    if filename.endswith('.tsv'):
        # Construct the full file path
        input_file = os.path.join(input_folder, filename)
        output_file = os.path.join(input_folder, f"{filename[:8]}_metadata.tsv")

        # # Read the TSV file into a DataFrame
        df = pd.read_csv(input_file, sep='\t')

        # The last two lines are showing where the raw data tables start and end
        # they do not contain any info we can use right now
        # all the relevant info can be found parsed in the Raw Data Dataframes
        df = df.iloc[:-2]

        # FIX column names
        df.columns = range(df.shape[1])
        # columnames = df.columns
        # print(columnames)
        # Set the first column as row names
        df = df.set_index(df[0])
        df = df.iloc[:, 1:]
        # rownames = df.index
        # print(rownames)

        # Remove all types of quotation marks
        df = df.map(lambda x: re.sub(r'["\'"]+', '', x) if isinstance(x, str) and not pd.isna(x) else x)

        # After running the code multiple times, only one line in the output could not be
        # split properly, so I went with this method, because notheing else worked
        # For rowname "Series_sample_id", split by SPACE DELIMITER
        # value = df.loc['Series_sample_id', df.columns[0]]  # Access using row name and column index
        # if isinstance(value, str):
        #         split_values = value.split()
        #         df.loc['Series_sample_id', df.columns[0]] = split_values  # Update using row name and column index

        ##############################################
        # I want to comeback after and maybe try and fix this
        ##############################################

        # Save the result as a TSV file
        df.to_csv(output_file, sep='\t')

        print(f"Metadata for {filename} has been saved to {output_file}")

Metadata for GSE85543_series_matrix_parsed_metadata.tsv has been saved to /content/sample_data/THESIS/GSE85543_metadata.tsv
Metadata for GSE85544_series_matrix_parsed_metadata.tsv has been saved to /content/sample_data/THESIS/GSE85544_metadata.tsv


Was not able to split the Sample_series_id values for now
If I ever need to use the Sample_series_id, the alternative is the Sample_Geo_Accession